In [ ]:
# Spark graphx analytics

# Mount the data from AWS S3
aws_bucket_name = "xxxx"
mount_name = "xxxx"

access_key = dbutils.secrets.get(scope = "aws", key = "aws-access-key")
secret_key = dbutils.secrets.get(scope = "aws", key = "aws-secret-key")
encoded_secret_key = secret_key.replace("/", "%2F")

dbutils.fs.mount("s3a://%s:%s@%s" % (access_key, encoded_secret_key, aws_bucket_name), "/mnt/%s" % mount_name)

In [ ]:
display(dbutils.fs.ls("/mnt/%s" % mount_name))

In [ ]:
df = spark.read.text("/mnt/%s/embeddings/NYT_train_df_nn.csv" % mount_name)

In [ ]:
df.show(2)

In [ ]:
import graphframes

In [ ]:
# Load the vertices and edges back.
#g2.vertices.write.parquet("dbfs:/mnt/xxxx/embeddings/lcc_vertices")
#g2.edges.write.parquet("dbfs:/mnt/xxxx/embeddings/lcc_edges")

sameV= sqlContext.read.parquet("dbfs:/mnt/xxxx/embeddings/span_lcc_vertices")
sameE = sqlContext.read.parquet("dbfs:/mnt/xxxx/embeddings/span_lcc_edges")

#sameV= sqlContext.read.parquet("dbfs:/mnt/xxxx/embeddings/graph_span_vertices")
#sameE = sqlContext.read.parquet("dbfs:/mnt/xxxx/embeddings/graph_span_edges")

In [ ]:
from graphframes import GraphFrame
g = GraphFrame(sameV, sameE)

In [ ]:
g.vertices.count()

In [ ]:
g.edges.count()

In [ ]:
sc.setCheckpointDir("/FileStore/tables/gem/chkPoint/")

In [ ]:
#triangles = g.triangleCount()

In [ ]:
#triangles.select('id', 'count').show(5)

In [ ]:
#triangles.coalesce(1) \
#      .write \
#      .option("header","true") \
#      .option("sep",",") \
#      .mode("overwrite") \
#      .csv("dbfs:/mnt/xxxx/embeddings/triangles")

In [ ]:
result = g.connectedComponents()

In [ ]:
result.select("id", "component").orderBy("component").show()

In [ ]:
result.select("component").distinct().count()

In [ ]:
result.coalesce(1) \
      .write \
      .option("header","true") \
      .option("sep",",") \
      .mode("overwrite") \
      .csv("dbfs:/mnt/xxxx/embeddings/span_largestConnectedComponents")

In [ ]:
degrees = g.degrees

In [ ]:
degrees.show()

In [ ]:
from pyspark.sql.functions import desc

In [ ]:
degrees.coalesce(1) \
      .write \
      .option("header","true") \
      .option("sep",",") \
      .mode("overwrite") \
      .csv("dbfs:/mnt/xxxx/embeddings/span_lcc_degrees")

#.csv("dbfs:/mnt/xxxx/embeddings/largest_cc_degrees")

In [ ]:
degrees.select("id", "degree").orderBy(desc("degree")).show()

In [ ]:
# Run PageRank until convergence to tolerance "tol".
pranks = g.pageRank(resetProbability=0.15, tol=0.01)

In [ ]:
pranks.vertices.select('id', 'pagerank').show(3)

In [ ]:
pranks.vertices.select('id').count()

In [ ]:
pranks.vertices.coalesce(1) \
      .write \
      .option("header","true") \
      .option("sep",",") \
      .mode("overwrite") \
      .csv("dbfs:/mnt/xxxx/embeddings/span_lcc_page_ranks")

In [ ]:
pranks.edges.select('dst', 'src', 'weight').show(2)

In [ ]:
#pranks.edges.coalesce(1) \
#      .write \
#      .option("header","true") \
#      .option("sep",",") \
#      .mode("overwrite") \
#      .csv("dbfs:/mnt/xxxx/embeddings/page_ranks_edge_weights")

In [ ]:
landmarks_samples = g.vertices.sample(0.0001).select('id').rdd.map(lambda r: r[0]).collect()

In [ ]:
landmarks_samples

In [ ]:
shpaths = g.shortestPaths(landmarks = landmarks_samples)

In [ ]:
shpaths.select('id', 'distances').show(4)

In [ ]:
shpaths.show(4)

In [ ]:
shpaths.coalesce(1) \
      .write \
      .json("dbfs:/mnt/xxxx/embeddings/span_lcc_shortest_paths_randomPoint10Percent")